<h1><center>Walmart</center></h1>
<h2><center>Capacitated Facility Location</center></h2>

Developed by: Daniel A. Zuniga Vazquez\
Date: 10/02/2021\
Version 1.0.0 

Sets and indices\
$I$: Set of facilities, indexed by $i$.\
$J$: Set of customers, indexed by $j$

Parameters\
$c_{ij}$: Cost of facility $i$ to supply to customer $j$.\
$d_j$:Demand of customer $j$.\
$f_i$:Cost of adding facility $i$.\
$u_i$:Capacity of facility $i$.

Variables\
$x_i$: Binary variables that is 1 if facility $i$ is assigned and 0 otherwise.\
$y_{ij}$: Fraction of demand supplied from facility $i$ to customer $j$.

Problem Formulation\
$
\begin{align}
\min_{\pmb{x},\pmb{y}}~& \sum_{i \in I} \sum_{ j \in J} c_{ij} d_j y_{ij} + \sum_{i \in I} f_i x_i & \text{Objective function}\\
    \text{s.t. } &  \sum_{i \in I} y_{ij} = 1,~\forall j \in J  & \text{2.1a - Satisfied fraction of demand}\\
    & \sum_{j \in J} d_j y_{ij} \leq u_i x_i,~\forall i \in I & \text{2.1b - Facility capacity}\\
    & y_{ij} \leq 0, x_i \in \{0,1\},~\forall i \in I, \forall j \in J & \text{2.1c - Domain}
\end{align}
$


In [ ]:
# For purposes of this Examples tutorial, we will restart the kernel for each problem
import os
os._exit(00)

In [1]:
# To plot interactable figures and print in notebook
%matplotlib notebook

# Libraries
import pandas as pd                #data manipulation and analysis library
import numpy as np                 #array and matrices library
from docplex.mp.model import Model #cplex library

from IPython.core.debugger import set_trace #Library to help with debgging      Use set_trace()   if breakpoint is needed
# Debugging commands ONLY use first letter
# n(ext) execute the current statement (step over)
# s(tep) execute and step into function
# r(eturn) continue execution until the current function returns
# c(ontinue) continue execution until a breakpoint is encountered
# u(p) move one level up in the stack trace
# d(own) move one level down in the stack trace

In [2]:
# Reading parameters from external TXT
#*******************************************
# NOTE: for this purpose, a dummy initial row of zeros is added in the txt file matching the length of the last column

# File name must be in the same directory as the python file, or you can use the complete path if not
fileName = 'Walmart_CapacitatedFacilityLocation.txt'

# Identify the length of the column
columns = max(len(l.split()) for l in open(fileName))

# Use panda read_table function to read all the values in a table
tableValues = pd.read_table(fileName, 
                    delim_whitespace=True, 
                    header=None, 
                    usecols=range(columns), 
                    engine='python')

# Transform the table values into a numpy array, missing values will be identified as NaN just to complete the matrix
# Note that the dummy initial row of zeros is still there and integers will be read as float, so the integer conversion
# required
iData = np.array(tableValues)

# Declare parameters
I = int(iData[1][0])       # Set of facilities, cardinality
J = int(iData[2][0])        # Set of customers, cardinality

# c[i][j]: Cost of facility i to supply to customer j. 
c = {(i,j):int((iData[i+3][j])) for i in range(I) for j in range(J)} 

f = {(i):int((iData[I+3][i])) for i in range(I)}    # Cost of adding facility i.
d = {(j):int((iData[I+4][j])) for j in range(J)}    # Demand of customer j.
u = {(i):int((iData[I+5][i])) for i in range(I)}    # Capacity of facility i.

In [3]:
# Creates cplex model
mycplex = Model('CapacitatedFacilityLocation_PYTHON')   #Name of Model

{0: 3000, 1: 4000, 2: 5000}

In [7]:
# Declare variables

# Auxiliary array to create binary variable name
nameX = [(i) for i in range(I)] 
# x[i]: Binary variables that is 1 if facility i is assigned and 0 otherwise.
x = mycplex.binary_var_dict(nameX, name='x')

# Auxiliary array to create continuous variable name
nameY = [(i,j) for i in range(I) for j in range(J)]  
# y[i][j]: Fraction of demand supplied from facility i to customer j.
y = mycplex.continuous_var_dict(nameY, lb=0, ub=1, name='y',)


In [8]:
# Create objective

OBJ = 0
for i in range(I):
    OBJ += f[i]*x[i]
    for j in range(J):
        OBJ += c[i,j]* d[j] * y[i,j]

mycplex.minimize(OBJ) 

In [9]:
# Adding constraints

# Constraint 2.1a - Satisfied fraction of demand
for j in range(J):
    mycplex.add_constraint(sum(y[i,j] for i in range(I)) == 1)

# Constraint 2.1b - Facility capacity
for i in range(I):
    mycplex.add_constraint(sum(d[j] * y[i,j] for j in range(J)) - (u[i] * x[i]) <= 0)


In [10]:
#Export cplex model
mycplex.export_as_lp('CapacitatedFacilityLocation_PYTHON')  # exports model in lp format
mycplex.export_as_mps('CapacitatedFacilityLocation_PYTHON') # exports model in mps format

'CapacitatedFacilityLocation_PYTHON.mps'

In [11]:
# Solve Cplex model
solution = mycplex.solve(log_output = True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 9 rows, 21 columns, and 39 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Found incumbent of value 13443.000000 after 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 9 rows, 21 columns, and 39 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0               

In [12]:
# Print Solutions
print(solution)

solution for: CapacitatedFacilityLocation_PYTHON
objective: 7767
x_0=1
x_1=1
y_0_0=1.000
y_0_5=0.889
y_1_1=1.000
y_1_2=1.000
y_1_3=1.000
y_1_4=1.000
y_1_5=0.111



<h1><center>Walmart</center></h1>
<h2><center>Uncapacitated Facility Location</center></h2>

Developed by: Daniel A. Zuniga Vazquez\
Date: 10/02/2021\
Version 1.0.0 

Sets and indices\
$I$: Set of facilities, indexed by $i$.\
$J$: Set of customers, indexed by $j$

Parameters\
$c_{ij}$: Cost of facility $i$ to supply to customer $j$.\
$d_j$:Demand of customer $j$.\
$f_i$:Cost of adding facility $i$.\
$M$:Big M, i.e., sufficiently big number.

Variables\
$x_i$: Binary variables that is 1 if facility $i$ is assigned and 0 otherwise.\
$y_{ij}$: Fraction of demand supplied from facility $i$ to customer $j$.

Problem Formulation\
$
\begin{align}
\min_{\pmb{x},\pmb{y}}~& \sum_{i \in I} \sum_{ j \in J} c_{ij} d_j y_{ij} + \sum_{i \in I} f_i x_i & \text{Objective function}\\
    \text{s.t. } &  \sum_{i \in I} y_{ij} = 1,~\forall j \in J  & \text{2.1a - Satisfied fraction of demand}\\
    & \sum_{j \in J} d_j y_{ij} \leq M x_i,~\forall i \in I & \text{2.1b - Facility capacity}\\
    & y_{ij} \leq 0, x_i \in \{0,1\},~\forall i \in I, \forall j \in J & \text{2.1c - Domain}
\end{align}
$


In [ ]:
# For purposes of this Examples tutorial, we will restart the kernel for each problem
import os
os._exit(00)

In [1]:
# To plot interactable figures and print in notebook
%matplotlib notebook

# Libraries
import pandas as pd                #data manipulation and analysis library
import numpy as np                 #array and matrices library
from docplex.mp.model import Model #cplex library

from IPython.core.debugger import set_trace #Library to help with debgging      Use set_trace()   if breakpoint is needed
# Debugging commands ONLY use first letter
# n(ext) execute the current statement (step over)
# s(tep) execute and step into function
# r(eturn) continue execution until the current function returns
# c(ontinue) continue execution until a breakpoint is encountered
# u(p) move one level up in the stack trace
# d(own) move one level down in the stack trace

In [2]:
# Reading parameters from external TXT
#*******************************************
# NOTE: for this purpose, a dummy initial row of zeros is added in the txt file matching the length of the last column

# File name must be in the same directory as the python file, or you can use the complete path if not
fileName = 'Walmart_UncapacitatedFacilityLocation.txt'

# Identify the length of the column
columns = max(len(l.split()) for l in open(fileName))

# Use panda read_table function to read all the values in a table
tableValues = pd.read_table(fileName, 
                    delim_whitespace=True, 
                    header=None, 
                    usecols=range(columns), 
                    engine='python')

# Transform the table values into a numpy array, missing values will be identified as NaN just to complete the matrix
# Note that the dummy initial row of zeros is still there and integers will be read as float, so the integer conversion
# required
iData = np.array(tableValues)

# Declare parameters
I = int(iData[1][0])       # Set of facilities, cardinality
J = int(iData[2][0])        # Set of customers, cardinality

# c[i][j]: Cost of facility i to supply to customer j. 
c = {(i,j):int((iData[i+3][j])) for i in range(I) for j in range(J)} 

f = {(i):int((iData[I+3][i])) for i in range(I)}    # Cost of adding facility i.
d = {(j):int((iData[I+4][j])) for j in range(J)}    # Demand of customer j.
M = int(iData[I+5][0])    # Capacity of facility i.

In [3]:
# Creates cplex model
mycplex = Model('UncapacitatedFacilityLocation_PYTHON')   #Name of Model

In [4]:
# Declare variables

# Auxiliary array to create binary variable name
nameX = [(i) for i in range(I)] 
# x[i]: Binary variables that is 1 if facility i is assigned and 0 otherwise.
x = mycplex.binary_var_dict(nameX, name='x')

# Auxiliary array to create continuous variable name
nameY = [(i,j) for i in range(I) for j in range(J)]  
# y[i][j]: Fraction of demand supplied from facility i to customer j.
y = mycplex.continuous_var_dict(nameY, lb=0, ub=1, name='y',)

In [5]:
# Create objective

OBJ = 0
for i in range(I):
    OBJ += f[i]*x[i]
    for j in range(J):
        OBJ += c[i,j]* d[j] * y[i,j]

mycplex.minimize(OBJ) 

In [6]:
# Adding constraints

# Constraint 2.1a - Satisfied fraction of demand
for j in range(J):
    mycplex.add_constraint(sum(y[i,j] for i in range(I)) == 1)

# Constraint 2.1b - Facility capacity
for i in range(I):
    mycplex.add_constraint(sum(d[j] * y[i,j] for j in range(J)) - (M * x[i]) <= 0)


In [7]:
#Export cplex model
mycplex.export_as_lp('UncapacitatedFacilityLocation_PYTHON')  # exports model in lp format
mycplex.export_as_mps('UncapacitatedFacilityLocation_PYTHON') # exports model in mps format

'UncapacitatedFacilityLocation_PYTHON.mps'

In [8]:
# Solve Cplex model
solution = mycplex.solve(log_output = True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 4185.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve modified 3 coefficients.
Reduced MIP has 9 rows, 21 columns, and 39 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 9 rows, 21 columns, and 39 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt 

In [9]:
# Print Solutions
print(solution)

solution for: UncapacitatedFacilityLocation_PYTHON
objective: 4185
x_0=1
y_0_0=1.000
y_0_1=1.000
y_0_2=1.000
y_0_3=1.000
y_0_4=1.000
y_0_5=1.000

